In [1]:
from datetime import datetime

import dataset_utils as dataset
import torch as t
from ML_cup.pytorch import *
from ML_cup.dataset_utils import CupDataset

In [2]:
import os
import time

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
    torch.set_default_device(torch.device("cuda"))
print(f"Using device: {torch.cuda.current_device()}")
print(f"Available cpu count: {os.cpu_count()}")
dev_data = dataset.load_dataset("../data/ML-CUP24-TR.csv")
blind_data = dataset.load_dataset("../data/ML-CUP24-TS.csv")

# get a seed for the random state based on the current time
rand = int(time.time())
print(f"Random seed: {rand}")
np.random.seed(rand)
# Split the dev data into train and validation with k-fold cross validation and minibatch
train_loaders, val_loaders, dev_loader, test_loader = dataset.torch_k_fold(dataset=dev_data, folds=5, batch_size=1,
                                                                           random_state=rand)
blind_loader = DataLoader(CupDataset(blind_data, device=torch.device('cuda')), batch_size=None)

GPU is available
Using device: 0
Available cpu count: 16
Random seed: 1741017289


In [3]:
# Model Definitions
def MLP() -> t.nn.Module:
    return t.nn.Sequential(
        t.nn.Linear(12, 200),
        t.nn.Tanh(),
        t.nn.Linear(200, 200),
        t.nn.Tanh(),
        t.nn.Linear(200, 3)
    )


def MLPr() -> t.nn.Module:
    return t.nn.Sequential(
        t.nn.Linear(12, 200),
        t.nn.ReLU(),
        t.nn.Linear(200, 200),
        t.nn.ReLU(),
        t.nn.Linear(200, 3)
    )

In [ ]:
parameters = [
    {
        'optimizer': 'SGD',
        'lr': np.linspace(0.0005, 0.00025, 5),
        'weight_decay': np.linspace(0.0001, 0.00001, 5),
        'momentum': [0.9, 0.85, 0.8],
        'nesterov': [True, False]
    }
]

# perform grid search to find the best parameters
best_pytorch_paramss, best_models = grid_search(model_builder=MLP, parameters=parameters, random_seed=rand,
                                              train_loader=train_loaders,
                                              val_loader=val_loaders, max_epochs=300,
                                              scheduler=('', {}),
                                              stability_threshold=0.25, patience=10,
                                              clip=1.0,
                                              tensorboard_folder_base=f'runs/online/MLP/SGD/tanh/{rand}')

Avg Train MEE: 0.36825414242455734, Avg Val MEE: 0.8768522168509663, Avg Fluctuation: 0.002321981589941302
('SGD', {'lr': 0.00075, 'weight_decay': 5.5e-05, 'momentum': 0.9, 'nesterov': True}, 1.0)
Avg Train MEE: 0.3529301472408406, Avg Val MEE: 0.8610409125126898, Avg Fluctuation: 0.0023054787779223174
('SGD', {'lr': 0.00075, 'weight_decay': 0.0001, 'momentum': 0.9, 'nesterov': True}, 1.0)
Avg Train MEE: 0.35445258340565494, Avg Val MEE: 0.8571365180239081, Avg Fluctuation: 0.001339464165180705
('SGD', {'lr': 0.00075, 'weight_decay': 0.0001, 'momentum': 0.85, 'nesterov': True}, 1.0)
Avg Train MEE: 0.33749539519427346, Avg Val MEE: 0.837492194660008, Avg Fluctuation: 0.0020054516209488645
('SGD', {'lr': 0.00075, 'weight_decay': 5.5e-05, 'momentum': 0.85, 'nesterov': False}, 1.0)
Avg Train MEE: 0.32507067048645694, Avg Val MEE: 0.8231994131207465, Avg Fluctuation: 0.0012591345233209197
('SGD', {'lr': 0.00075, 'weight_decay': 5.5e-05, 'momentum': 0.8, 'nesterov': False}, 1.0)
Avg Train ME

In [ ]:
parameters = [
    {
        'optimizer': 'Adam',
        'lr': np.linspace(0.000025, 0.0000075, 10),
        'weight_decay': np.linspace(0.0001, 0.00001, 5),
    }]
# perform grid search to find the best parameters
best_pytorch_paramsa, best_modela = grid_search(model_builder=MLP, parameters=parameters, random_seed=rand,
                                              train_loader=train_loaders,
                                              val_loader=val_loaders, max_epochs=300,
                                              scheduler=('', {}),
                                              stability_threshold=0.25, patience=10,
                                              clip=1.0,
                                              tensorboard_folder_base=f'runs/online/MLP/Adam/tanh/{rand}')